In [1]:
# only needed for colab
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import os
# os.chdir('/content/drive/MyDrive/TaylorExpansionML')
os.chdir("..")

In [3]:
import numpy as np
import pickle
import sympy as sp
from sympy import sympify, srepr
import os
from icecream import ic
import random

from source.data_preparation import sympy_to_prefix, prefix_to_sympy, vectorize_ds, vectorize_sentence, pad_right
import functools

In [4]:
from tqdm.notebook import trange, tqdm

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow import keras
from tensorflow.keras import layers

2022-05-31 11:16:41.751222: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-31 11:16:41.753875: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-31 11:16:41.753885: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
from gensim.models import Word2Vec
from icecream import ic

In [7]:
load_data_from_scratch = False
export_data = False
load_full_data = True
check_prefix_recovery = False

In [8]:
if load_data_from_scratch:
    data_file = "data.nosync/data.txt"
    taylor_file = "data.nosync/data_taylor.txt"
    coeffs_file = "data.nosync/data_coeffs.txt"

    start = ["[start]"]
    end = ["[end]"]

    with open(data_file) as f:
        X = f.read().split("\n")
        X = np.array(X)
        X = X[:-1]   # somehow last entry is empty
        X = [sympify(xi) for xi in X]

    with open(taylor_file) as f:
        y_taylor = f.read().split("\n")
        y_taylor = np.array(y_taylor)
        y_taylor = y_taylor[:-1]
        y_taylor = [sympify(yi) for yi in y_taylor]

    with open(coeffs_file) as f:
        y_coeffs = f.read().split("\n")
        y_coeffs = y_coeffs[:-1]
        for i, y in enumerate(y_coeffs):
            y = sympify(y)
            # add start end tokens and remove whitespaces
            y_coeffs[i] = y

In [9]:
X_pickle_file_train = "data.nosync/data_train.pickle"
y_taylor_pickle_file_train = "data.nosync/data_taylor_train.pickle"
y_coeffs_pickle_file_train = "data.nosync/data_coeffs_train.pickle"

X_pickle_file_small_train = "data.nosync/data_train_small.pickle"
y_taylor_pickle_file_small_train = "data.nosync/data_taylor_train_small.pickle"
y_coeffs_pickle_file_small_train = "data.nosync/data_coeffs_train_small.pickle"

X_pickle_file_test = "data.nosync/data_test.pickle"
y_taylor_pickle_file_test = "data.nosync/data_taylor_test.pickle"
y_coeffs_pickle_file_test = "data.nosync/data_coeffs_test.pickle"

In [10]:
if export_data:
    with open(X_pickle_file_train, "wb") as f:
        pickle.dump(X[0:10_000], f)
    with open(y_taylor_pickle_file_train, "wb") as f:
        pickle.dump(y_taylor[0:10_000], f)
    with open(y_coeffs_pickle_file_train, "wb") as f:
        pickle.dump(y_coeffs[0:10_000], f)

    with open(X_pickle_file_small_train, "wb") as f:
        pickle.dump(X[0:100], f)
    with open(y_taylor_pickle_file_small_train, "wb") as f:
        pickle.dump(y_taylor[0:100], f)
    with open(y_coeffs_pickle_file_small_train, "wb") as f:
        pickle.dump(y_coeffs[0:100], f)

    with open(X_pickle_file_test, "wb") as f:
        pickle.dump(X[10_000:], f)
    with open(y_taylor_pickle_file_test, "wb") as f:
        pickle.dump(y_taylor[10_000:], f)
    with open(y_coeffs_pickle_file_test, "wb") as f:
        pickle.dump(y_coeffs[10_000:], f)

In [11]:
if (not load_data_from_scratch) & (not load_full_data):
    with open(X_pickle_file_small_train, "rb") as f:
        X_train = pickle.load(f)
    with open(y_taylor_pickle_file_small_train, "rb") as f:
        y_taylor_train = pickle.load(f)
    with open(y_coeffs_pickle_file_small_train, "rb") as f:
        y_coeffs_train = pickle.load(f)

    with open(X_pickle_file_test, "rb") as f:
        X_test = pickle.load(f)
    with open(y_taylor_pickle_file_test, "rb") as f:
        y_taylor_test = pickle.load(f)
    with open(y_coeffs_pickle_file_test, "rb") as f:
        y_coeffs_test = pickle.load(f)

elif load_full_data:
    with open(X_pickle_file_train, "rb") as f:
        X_train = pickle.load(f)
    with open(y_taylor_pickle_file_train, "rb") as f:
        y_taylor_train = pickle.load(f)
    with open(y_coeffs_pickle_file_train, "rb") as f:
        y_coeffs_train = pickle.load(f)
    with open(X_pickle_file_test, "rb") as f:
        X_test = pickle.load(f)
    with open(y_taylor_pickle_file_test, "rb") as f:
        y_taylor_test = pickle.load(f)
    with open(y_coeffs_pickle_file_test, "rb") as f:
        y_coeffs_test = pickle.load(f)

In [12]:
ic(len(X_train))
ic(len(X_test))

ic(len(y_taylor_train))
ic(len(y_taylor_test))

ic| len(X_train): 10000
ic| len(X_test): 1000
ic| len(y_taylor_train): 10000
ic| len(y_taylor_test): 1000


1000

In [13]:
X_prefix_train = [sympy_to_prefix(x) for x in X_train]
y_taylor_prefix_train = [sympy_to_prefix(y) for y in y_taylor_train]

X_prefix_test = [sympy_to_prefix(x) for x in X_test]
y_taylor_prefix_test = [sympy_to_prefix(y) for y in y_taylor_test]

In [14]:
if check_prefix_recovery:
    X_prefix_train_recovered = [prefix_to_sympy(x) for x in X_prefix_train]
    y_taylor_prefix_train_recovered = [prefix_to_sympy(y) for y in y_taylor_prefix_train]

    X_prefix_test_recovered = [prefix_to_sympy(x) for x in X_prefix_test]
    y_taylor_prefix_test_recovered = [prefix_to_sympy(y) for y in y_taylor_prefix_test]


    assert X_prefix_train_recovered == X_train
    assert y_taylor_prefix_train_recovered == y_taylor_train
    print("All train data correctly recovered with `prefix_to_sympy`")

    assert X_prefix_test_recovered == X_test
    assert y_taylor_prefix_test_recovered == y_taylor_test

    print("All test data correctly recovered with `prefix_to_sympy`")

In [15]:
def clean_data(X, y, max_len=200, del_manual_idxs=[]):
    """ Clean up data
    - manually remove indices from `del_manual_idxs`
    - remove where prefix notation is longer than 200
    """
    previous_length = len(X)

    X = [val for i, val in enumerate(X) if i not in del_manual_idxs]
    y = [val for i, val in enumerate(y) if i not in del_manual_idxs]

    X_too_long_idxs = too_long(X, max_len)
    X = [val for i, val in enumerate(X) if i not in X_too_long_idxs]
    y = [val for i, val in enumerate(y) if i not in X_too_long_idxs]

    y_too_long_idxs = too_long(y, max_len)
    X = [val for i, val in enumerate(X) if i not in y_too_long_idxs]
    y = [val for i, val in enumerate(y) if i not in y_too_long_idxs]

    total_removed = len(del_manual_idxs) + len(X_too_long_idxs) + len(y_too_long_idxs)
    del_percentage = total_removed / previous_length
    print("Deleted ", total_removed, "elements", "of", previous_length)
    print("That's ", round(del_percentage*100, 1), "%")

    return (X, y)


def too_long(data, max_len):
    """return indices where len(data)>max_len"""
    return [i for (i, x) in enumerate(data) if len(x)>max_len]

In [16]:
del_manual_idxs = [
    2775, # too long, recusion time out
]
max_len = 200
print("train:")
X_prefix_train, y_taylor_prefix_train = clean_data(X_prefix_train, y_taylor_prefix_train, del_manual_idxs=del_manual_idxs, max_len=max_len)

print("test:")
X_prefix_test, y_taylor_prefix_test = clean_data(X_prefix_test, y_taylor_prefix_test, max_len=max_len)

train:
Deleted  1867 elements of 10000
That's  18.7 %
test:
Deleted  179 elements of 1000
That's  17.9 %


In [17]:
len(X_prefix_train)

8133

In [18]:
val_split_idx = 7933
X_prefix_val = X_prefix_train[val_split_idx:]
X_prefix_train = X_prefix_train[:val_split_idx]
y_taylor_prefix_val = y_taylor_prefix_train[val_split_idx:]
y_taylor_prefix_train = y_taylor_prefix_train[:val_split_idx]

# Vectorization

In [19]:
def make_one_string(word_arr, start_end=False):
    ret = functools.reduce(lambda a, b: a+ " " + b, word_arr)
    if start_end:
        ret = "[start]" + " " + ret + " " + "[end]"
    return ret

In [20]:
from tensorflow.keras.layers import TextVectorization

In [21]:
# vocab_size = 1000
sequence_length = 200

numbers_tokens = ['int ', 's+', 's-'] +[str(i) for i in range(0,10)]
numbers_tokens = make_one_string(numbers_tokens)

In [22]:
X_vectorize_layer = TextVectorization(
    # max_tokens=vocab_size,
    standardize=None,
    split='whitespace',
    output_mode='int',
    output_sequence_length=sequence_length)

y_vectorize_layer = TextVectorization(
    # max_tokens=vocab_size,
    standardize=None,
    split='whitespace',
    output_mode='int',
    output_sequence_length=sequence_length+1)

2022-05-31 11:17:11.338191: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-31 11:17:11.338333: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-31 11:17:11.338360: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-05-31 11:17:11.338378: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-05-31 11:17:11.338397: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [23]:
X_prefix_train_str = [make_one_string(x) for x in X_prefix_train]
y_prefix_train_str = [make_one_string(y, start_end=True) for y in y_taylor_prefix_train]

X_prefix_val_str = [make_one_string(x) for x in X_prefix_val]
y_prefix_val_str = [make_one_string(y, start_end=True) for y in y_taylor_prefix_val]

X_prefix_test_str = [make_one_string(x) for x in X_prefix_test]
y_prefix_test_str = [make_one_string(y, start_end=True) for y in y_taylor_prefix_test]

In [24]:
X_prefix_train_str[0]

'cosh add a cos mul int s+ 2 x'

In [25]:
y_prefix_train_str[0]

'[start] add mul pow x int s+ 4 add mul int s+ 2 cosh add int s+ 1 a mul mul int s+ 2 pow int s+ 3 int s- 1 sinh add int s+ 1 a add mul int s- 2 mul pow x int s+ 2 sinh add int s+ 1 a cosh add int s+ 1 a [end]'

In [26]:
numbers = [str(i) for i in range(0,10)]
numbers = 'int s+ s_ ' + make_one_string(numbers)
X_vectorize_layer.adapt([numbers]+X_prefix_train_str)
y_vectorize_layer.adapt([numbers]+y_prefix_train_str)

In [27]:
X_enc0 = X_vectorize_layer(X_prefix_train_str[0:1])
X_enc0

<tf.Tensor: shape=(1, 200), dtype=int64, numpy=
array([[11,  9, 20, 12,  4,  3,  6,  8,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0]])>

In [28]:
y_enc0 = y_vectorize_layer(y_prefix_train_str[0:1])
y_enc0

<tf.Tensor: shape=(1, 201), dtype=int64, numpy=
array([[13,  8,  4,  6, 10,  2,  3, 11,  8,  4,  2,  3,  9, 19,  8,  2,
         3,  5, 21,  4,  4,  2,  3,  9,  6,  2,  3, 12,  2,  7,  5, 16,
         8,  2,  3,  5, 21,  8,  4,  2,  7,  9,  4,  6, 10,  2,  3,  9,
        16,  8,  2,  3,  5, 21, 19,  8,  2,  3,  5, 21, 14,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0]])>

In [29]:
X_vocabulary = np.array(X_vectorize_layer.get_vocabulary())
y_vocabulary = np.array(y_vectorize_layer.get_vocabulary())

In [30]:
def unvectorize(sentence, vocab):
    return list(vocab[tuple(sentence)])

In [31]:
X_train_vec = X_vectorize_layer(X_prefix_train_str)
y_train_vec = y_vectorize_layer(y_prefix_train_str)

X_val_vec = X_vectorize_layer(X_prefix_val_str)
y_val_vec = y_vectorize_layer(y_prefix_val_str)

X_test_vec = X_vectorize_layer(X_prefix_test_str)
y_test_vec = y_vectorize_layer(y_prefix_test_str)

In [32]:
batch_size=128

def format_dataset(X, y):
    return ({"encoder_inputs": X, "decoder_inputs": y[:, :-1],}, y[:, 1:])

def make_dataset(X, y):
    dataset = format_dataset(X,y)
    dataset = tf.data.Dataset.from_tensor_slices(dataset)
    dataset = dataset.batch(batch_size)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(X_train_vec, y_train_vec)
val_ds = make_dataset(X_val_vec, y_val_vec)

In [33]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (128, 200)
inputs["decoder_inputs"].shape: (128, 200)
targets.shape: (128, 200)


2022-05-31 11:17:11.918017: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


# Transformer

In [34]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [35]:
pip install keras_tuner

Note: you may need to restart the kernel to use updated packages.


In [36]:
import keras_tuner as kt

In [37]:
def model_builder(hp):
    # embed_dim = 256
    # latent_dim = 512*2
    # num_heads = 7
    vocab_size = 200

    embed_dim = hp.Int('embed_dim', min_value=256, max_value=512, step=56)
    num_heads = hp.Int('num_heads', min_value=4, max_value=10, step=1)
    latent_dim = hp.Int('latent_dim', min_value=256, max_value=512*4, step=256)

    
    encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
    x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
    encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
    encoder = keras.Model(encoder_inputs, encoder_outputs)
    
    decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
    encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
    x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
    x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
    x = layers.Dropout(0.5)(x)
    decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
    decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)
    
    decoder_outputs = decoder([decoder_inputs, encoder_outputs])
    transformer = keras.Model(
        [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
    )
    transformer.compile(
        "rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return transformer

In [38]:
tuner = kt.Hyperband(model_builder,
                    overwrite=True,
                    objective="val_loss",
                    max_epochs=200,
                    factor=3,
                    directory="models/2022-05-31-TaylorModelPrefix",
                    project_name="2022-05-31-TaylorModelPrefix")

In [39]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=50)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=20, min_lr=1e-5) 

tuner.search(train_ds,
             validation_data=val_ds,
             epochs=200,
             callbacks=[
                 #stop_early,
                keras.callbacks.TensorBoard("logs/2022-05-31-TaylorModelPrefix")
             ],
             )


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
256               |?                 |embed_dim
7                 |?                 |num_heads
1792              |?                 |latent_dim
3                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
4                 |?                 |tuner/bracket
0                 |?                 |tuner/round

Layer PositionalEmbedding has arguments ['self', 'sequence_length', 'vocab_size', 'embed_dim']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config
Epoch 1/3


/home/boggog/anaconda3/lib/python3.9/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


 7/62 [==>...........................] - ETA: 4:01 - loss: 1.6876 - accuracy: 0.1389

KeyboardInterrupt: 

In [40]:
tmp_hps = kt.HyperParameters()
tmp_hps.values = {
    'embed_dim': 200,
    'num_heads': 5,
    'latent_dim': 512,
           }

In [41]:
model = model_builder(tmp_hps)
model.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding_2 (Positi  (None, None, 200)   80000       ['encoder_inputs[0][0]']         
 onalEmbedding)                                                                                   
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder_1 (Transfo  (None, None, 200)   1009512     ['positional_embedding_

In [43]:
model.fit(train_ds, epochs=2, validation_data=val_ds)

Epoch 1/2
62/62 [==============================] - 122s 2s/step - loss: 0.6221 - accuracy: 0.4277 - val_loss: 0.4751 - val_accuracy: 0.5049
Epoch 2/2
62/62 [==============================] - 114s 2s/step - loss: 0.4131 - accuracy: 0.5193 - val_loss: 0.4325 - val_accuracy: 0.5341


In [44]:
model.save_weights("models/test_model_weights.h5")

In [45]:
model_reloaded = model_builder(tmp_hps)
model_reloaded.load_weights("models/test_model_weights.h5")

# Testing

In [52]:
max_decoded_sentence_length = 200
def apply_transformer(X, y_vectorize_layer):
    decoded_sentence = ["[start]"]
    vocab = y_vectorize_layer.get_vocabulary()

    # for i in range(max_decoded_sentence_length):
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = y_vectorize_layer(decoded_sentence)[:,0:max_decoded_sentence_length]
        y_pred = model_reloaded([X, tokenized_target_sentence])[0,i,:]
        y_index = np.argmax(y_pred)
        y = vocab[y_index]
        ic(y)
        decoded_sentence[0] = decoded_sentence[0] + " " + y

        if y == "[end]":
            break
    return decoded_sentence
    # return  decoded_sentence

In [53]:
y_taylor_prefix_train[0]

['add',
 'mul',
 'pow',
 'x',
 'int',
 's+',
 '4',
 'add',
 'mul',
 'int',
 's+',
 '2',
 'cosh',
 'add',
 'int',
 's+',
 '1',
 'a',
 'mul',
 'mul',
 'int',
 's+',
 '2',
 'pow',
 'int',
 's+',
 '3',
 'int',
 's-',
 '1',
 'sinh',
 'add',
 'int',
 's+',
 '1',
 'a',
 'add',
 'mul',
 'int',
 's-',
 '2',
 'mul',
 'pow',
 'x',
 'int',
 's+',
 '2',
 'sinh',
 'add',
 'int',
 's+',
 '1',
 'a',
 'cosh',
 'add',
 'int',
 's+',
 '1',
 'a']

In [54]:
apply_transformer(X_train_vec[0:1], y_vectorize_layer)

ic| y: 'add'
ic| y: 'mul'
ic| y: 'a'
ic| y: 'cos'
ic| y: 'a'
ic| y: 'mul'
ic| y: 'mul'
ic| y: 'int'
ic| y: 's+'
ic| y: '1'
ic| y: 'pow'
ic| y: 'x'
ic| y: 'int'
ic| y: 's+'
ic| y: '1'
ic| y: 'pow'
ic| y: 'x'
ic| y: 'int'
ic| y: 's+'
ic| y: '3'
ic| y: '[end]'


['[start] add mul a cos a mul mul int s+ 1 pow x int s+ 1 pow x int s+ 3 [end]']